In [1]:
# import packages
import requests
from bs4 import BeautifulSoup
from requests_html import AsyncHTMLSession


# Loading Libraries

In [204]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.action_chains import ActionChains
import time
import pandas as pd
import csv
import json

# Initiate Driver to Scroll through page

In [217]:
driver = webdriver.Chrome('/Users/tracynguyen/Applications/chromedriver')
# url = 'https://shop.lululemon.com/c/women/_/N-7vf'
url = 'https://shop.lululemon.com/c/women-maintops/_/N-815'

driver.get(url)  # open WMTM page
i = 0
while True:
    try:
        driver.execute_script('window.scrollTo(0, document.body.scrollHeight + 1000);')  
        time.sleep(15)  
        button = driver.find_element_by_xpath("//span[text()='View more products']")
        actions = ActionChains(driver)
        actions.move_to_element(button).perform()  # move button into view
        button.click()  
    
    except NoSuchElementException:  # all products loaded
        break
       
# driver.page_source  # return HTML text

## Get unique product urls
- ignore different colors

In [218]:
product_urls = [(i.get_attribute('href')).split("?", 1)[0] for i in driver.find_elements_by_xpath('.//div[@class="product-tile"]//a')]
product_urls = list(dict.fromkeys(product_urls))
product_urls

['https://shop.lululemon.com/p/tops-long-sleeve/Swiftly-Tech-LS-2/_/prod9750543',
 'https://shop.lululemon.com/p/women-tanks/Align-Tank-Diamond-Dye/_/prod10000326',
 'https://shop.lululemon.com/p/women-tanks/Align-Tank/_/prod9600539',
 'https://shop.lululemon.com/p/women-tanks/Blissful-Bend-Tank-Top/_/prod10190150',
 'https://shop.lululemon.com/p/women-tanks/Gather-and-Grow-Tank/_/prod10400588',
 'https://shop.lululemon.com/p/tops-long-sleeve/Gather-and-Grow-Long-Sleeve/_/prod10390404',
 'https://shop.lululemon.com/p/tops-long-sleeve/Ever-Ready-Long-Sleeve/_/prod9960686',
 'https://shop.lululemon.com/p/tops-long-sleeve/Back-In-Action-Ls/_/prod8555454',
 'https://shop.lululemon.com/p/tops-long-sleeve/LA-Back-in-Action-Long-Sleeve/_/prod9750526',
 'https://shop.lululemon.com/p/tops-long-sleeve/Rest-Less-Pullover/_/prod5020032',
 'https://shop.lululemon.com/p/womens-t-shirts/Love-Crew-III/_/prod8690612',
 'https://shop.lululemon.com/p/women-tanks/Power-Pivot-Tank-Rib/_/prod9750455',
 'htt

In [219]:
len(product_urls)

163

## Get product attributes

In [220]:
product_attributes = [ i.get_attribute('data-lulu-attributes') for i in driver.find_elements_by_xpath('.//h3[@class="product-tile__product-name lll-text-body-1"]//a')]
product_attributes

['{"type":"product","product":{"name":"Swiftly%20Tech%20Long%20Sleeve%202%2E0","skuID":"us_122876068","productID":"prod9750543","categoryUnifiedID":"swiftly-tech-ls-2","color":"45857","deliveryType":"ship"},"attributes":{"component":"product-list","displayType":"label","position":1},"version":"2.1"}',
 '{"type":"product","product":{"name":"lululemon%20Align%E2%84%A2%20Tank%20%2ADiamond%20Dye","skuID":"us_124975727","productID":"prod10000326","categoryUnifiedID":"align-tank-diamond-dye","color":"47168","deliveryType":"ship"},"attributes":{"component":"product-list","displayType":"label","position":2},"version":"2.1"}',
 '{"type":"product","product":{"name":"lululemon%20Align%E2%84%A2%20Tank","skuID":"us_127450385","productID":"prod9600539","categoryUnifiedID":"align-tank","color":"8313","deliveryType":"ship"},"attributes":{"component":"product-list","displayType":"label","position":4},"version":"2.1"}',
 '{"type":"product","product":{"name":"Blissful%20Bend%20Tank%20Top","skuID":"us_124

## Get product name

In [221]:
product_name = [ urllib.parse.unquote(json.loads(i.get_attribute('data-lulu-attributes'))['product']['name']) for i in driver.find_elements_by_xpath('.//h3[@class="product-tile__product-name lll-text-body-1"]//a')]
product_name

['Swiftly Tech Long Sleeve 2.0',
 'lululemon Align™ Tank *Diamond Dye',
 'lululemon Align™ Tank',
 'Blissful Bend Tank Top',
 'Gather and Grow Tank',
 'Gather and Grow Long Sleeve',
 'Ever Ready Long Sleeve',
 'Back In Action Long Sleeve',
 'LA Back in Action Long Sleeve',
 'Rest Less Pullover',
 'Love Crew III',
 'Power Pivot Tank *Rib',
 'Swiftly Tech Racerback 2.0',
 'Swiftly Tech Long Sleeve 2.0 *Race Length',
 'Swiftly Tech Long Sleeve 2.0 *MultiDye',
 'Rise and Run Short Sleeve',
 'Long Distance Short Sleeve',
 'All Yours Tee',
 'Time To Restore Short Sleeve',
 'Ebb To Street Tank *Shine',
 'Back in Action Long Sleeve *Rulu',
 'LA Split Shoulder Muscle Tank',
 'LA Mock Neck Layered Crop Tank',
 'Swiftly Tech Short Sleeve 2.0',
 'Love Tank *Pleated',
 'All Yours Tank',
 'Ebb To Street Tank *Light Support For B/C Cup',
 'All Tied Up Tank',
 'Cool Racerback II *Nulu',
 'Sculpt Tank',
 'Love Tee V',
 'Run Briskly 1/2 Zip',
 'Cool Racerback II',
 'Swiftly Breathe Long Sleeve Squad',
 

## Get product price

In [222]:
price = [i.get_attribute("innerText").replace("\xa0\n", "").replace("\xa0-\xa0", "-") for i in driver.find_elements_by_xpath('.//span[@class="price-1SDQy price"]')]
price

['$78',
 '$64',
 '$58',
 '$68',
 '$54',
 '$80',
 '$58',
 '$58',
 '$58',
 '$88',
 '$48',
 '$58',
 '$58',
 '$78',
 '$88',
 '$68',
 '$58',
 '$58',
 '$58',
 '$68',
 '$88',
 '$58',
 '$68',
 '$68',
 '$38',
 '$44',
 '$58',
 '$44',
 '$48',
 '$58',
 '$48',
 '$98',
 '$48',
 '$78',
 '$68',
 '$48',
 '$78',
 '$68',
 '$52',
 '$58',
 '$68',
 '$54',
 '$68',
 '$58',
 '$58',
 '$88',
 '$58',
 '$48',
 '$78',
 '$52',
 '$78',
 '$44',
 '$58',
 '$58',
 '$68',
 '$78',
 '$48',
 '$88',
 '$58',
 '$68',
 '$68',
 '$58',
 '$78',
 '$48',
 '$88',
 '$52',
 '$88',
 '$48',
 '$48',
 '$88',
 '$65',
 '$68',
 '$58',
 '$58',
 '$58',
 '$58',
 '$48',
 '$58',
 '$78',
 '$58',
 '$78',
 '$58',
 '$78',
 '$58',
 '$38',
 '$68',
 '$68',
 '$44',
 '$58',
 '$58',
 '$58',
 '$48',
 '$58',
 '$68',
 '$78',
 '$78',
 '$78',
 'Sale Price$39-$59Regular Price$68',
 'Sale Price$64Regular Price$78',
 'Sale Price$29Regular Price$44',
 'Sale Price$39Regular Price$58',
 'Sale Price$34-$39Regular Price$48',
 'Sale Price$64Regular Price$78',
 'Sale Price

# Make dataframe and clean it up

In [223]:
df = pd.DataFrame()
df['product_name'] = product_name
df['price'] = price
df['url'] = product_urls

new = df["price"].str.split("Regular Price", n = 1, expand = True)
df["current_price"] = new[0].str.replace('Sale Price','')
df["old_price"]= new[1] 

df = df.drop(['price'], axis = 1)
df

,product_name,url,current_price,old_price
0,Swiftly Tech Long Sleeve 2.0,https://shop.lululemon.com/p/tops-long-sleeve/...,$78,None
1,lululemon Align™ Tank *Diamond Dye,https://shop.lululemon.com/p/women-tanks/Align...,$64,None
2,lululemon Align™ Tank,https://shop.lululemon.com/p/women-tanks/Align...,$58,None
3,Blissful Bend Tank Top,https://shop.lululemon.com/p/women-tanks/Bliss...,$68,None
4,Gather and Grow Tank,https://shop.lululemon.com/p/women-tanks/Gathe...,$54,None
...,...,...,...,...
158,Full Day Ahead Shirt,https://shop.lululemon.com/p/tops-long-sleeve/...,$29,$98
159,Sheer Will Tank *Pulse,https://shop.lululemon.com/p/women-tanks/Sheer...,$29,$58
160,Stronger as One Muscle Tank,https://shop.lululemon.com/p/women-tanks/Stron...,$39,$58
161,Dynamic Days Tank,https://shop.lululemon.com/p/women-tanks/Dynam...,$49,$68


# Review of each url

In [256]:
tmp_urls = product_urls[0:2]
tmp_urls

['https://shop.lululemon.com/p/tops-long-sleeve/Swiftly-Tech-LS-2/_/prod9750543',
 'https://shop.lululemon.com/p/women-tanks/Align-Tank-Diamond-Dye/_/prod10000326']

In [262]:
driver.quit()
driver = webdriver.Chrome('/Users/tracynguyen/Applications/chromedriver')
product_index = 1
for item in tmp_urls:
    print(item)
    print('Product #' + str(product_index))
    product_index = product_index + 1
    review_index = 1
    driver.get(item)

    try:
        button = driver.find_element_by_xpath("//span[text()='Reviews']")
        actions = ActionChains(driver)
        actions.move_to_element(button).perform()  # move button into view
        button.click()  
#         review_button = driver.find_element_by_xpath('//a[@class="customer-engagement-extensions__button reviews-link"]')
#         review_button.click()
    except:
        driver.refresh()
        review_button = driver.find_element_by_xpath('//a[@class="customer-engagement-extensions__button reviews-link"]')
        review_button.click()
    time.sleep(5)

    product = driver.find_element_by_xpath('//h1[@class="pdp-title"]/div').text
    stars = [i.get_attribute("data-bv-histogram-rating-value") for i in driver.find_elements_by_xpath('.//div[@class="bv-inline-histogram-ratings-bar"]')]
    material = [i.get_attribute("innerHTML").replace(",", "").strip() for i in driver.find_elements_by_xpath('.//dd[@class="product-education-accordions__attributes__item__list"]')]
    
#     productCategory = driver.find_element_by_xpath('//*[@id="main-content"]/div/section/div/ul/li[2]/a').text
#     productCategoryDetail = driver.find_element_by_xpath('//*[@id="main-content"]/div/section/div/ul/li[3]/a').text
#     print(productCategory, productCategoryDetail)

https://shop.lululemon.com/p/tops-long-sleeve/Swiftly-Tech-LS-2/_/prod9750543
Product #1
https://shop.lululemon.com/p/women-tanks/Align-Tank-Diamond-Dye/_/prod10000326
Product #2


In [276]:
stars = [i.get_attribute("data-bv-histogram-rating-value") for i in driver.find_elements_by_xpath('.//div[@class="bv-inline-histogram-ratings-bar"]')]
stars

['5', '4', '3', '2', '1']

In [277]:
count = [i.get_attribute("data-bv-histogram-rating-count") for i in driver.find_elements_by_xpath('.//div[@class="bv-inline-histogram-ratings-bar"]')]
count

['1052', '279', '160', '90', '65']

In [311]:
material = [i.get_attribute("innerHTML").replace(",", "").strip() for i in driver.find_elements_by_xpath('.//dd[@class="product-education-accordions__attributes__item__list"]')]
material

['89% Nylon', '11% Elastane', '80% Nylon', '20% Lycra® elastane']

In [312]:
productCategoryDetail = driver.find_element_by_xpath('//*[@id="main-content"]/div/section/div/ul/li[3]/a').text
productCategoryDetail

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="main-content"]/div/section/div/ul/li[3]/a"}
  (Session info: chrome=88.0.4324.96)
